- 学修番号: 22745102
- 氏名: 岩間和也

# 課題 2.2　クラスタ分析

課題 2.2 クラスタ分析は、各自の興味に沿ったデータセットを取得し、分析してください。
クラスタ分析は、教師なし学習であり、正解ラベルはありません。
結果の解釈にはデータの理解が必要となりますので、興味があるデータについて分析してみてください。

分析は、以下の 2.2.1 ~ 2.2.4 について行ってください。

- 2.2.1 データセットの取得
- 2.2.2 階層的クラスタ分析
- 2.2.3 非階層的クラスタ分析
- 2.2.4 考察

### 2.2.1 データセットの取得、読み込み

2.2.1 では以下を行うこと。

- データセットの取得方法を記載
  - 取得、実行し、結果を確認できるように記載すること
- データセットの説明を記載
  - 処理内容を理解できるように説明を記載すること
- データセットの読み込み
  - （必要ならば）処理対象となるように加工

In [3]:
import pandas as pd
import numpy as np
data = pd.read_csv('data/taxis.csv')

# 不要なものの削除
data.drop(['pickup', 'dropoff', 'total'], axis=1)

# 欠損値の合計
print(data.isna().sum())

data = data.dropna()

# 確認
data

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
0,2019-03-23 20:21:09,2019-03-23 20:27:24,1,1.60,7.0,2.15,0.0,12.95,yellow,credit card,Lenox Hill West,UN/Turtle Bay South,Manhattan,Manhattan
1,2019-03-04 16:11:55,2019-03-04 16:19:00,1,0.79,5.0,0.00,0.0,9.30,yellow,cash,Upper West Side South,Upper West Side South,Manhattan,Manhattan
2,2019-03-27 17:53:01,2019-03-27 18:00:25,1,1.37,7.5,2.36,0.0,14.16,yellow,credit card,Alphabet City,West Village,Manhattan,Manhattan
3,2019-03-10 01:23:59,2019-03-10 01:49:51,1,7.70,27.0,6.15,0.0,36.95,yellow,credit card,Hudson Sq,Yorkville West,Manhattan,Manhattan
4,2019-03-30 13:27:42,2019-03-30 13:37:14,3,2.16,9.0,1.10,0.0,13.40,yellow,credit card,Midtown East,Yorkville West,Manhattan,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6428,2019-03-31 09:51:53,2019-03-31 09:55:27,1,0.75,4.5,1.06,0.0,6.36,green,credit card,East Harlem North,Central Harlem North,Manhattan,Manhattan
6429,2019-03-31 17:38:00,2019-03-31 18:34:23,1,18.74,58.0,0.00,0.0,58.80,green,credit card,Jamaica,East Concourse/Concourse Village,Queens,Bronx
6430,2019-03-23 22:55:18,2019-03-23 23:14:25,1,4.14,16.0,0.00,0.0,17.30,green,cash,Crown Heights North,Bushwick North,Brooklyn,Brooklyn
6431,2019-03-04 10:09:25,2019-03-04 10:14:29,1,1.12,6.0,0.00,0.0,6.80,green,credit card,East New York,East Flatbush/Remsen Village,Brooklyn,Brooklyn


In [9]:
X = data[['distance']]
Y = data.fare

# 線形単回帰モデルの作成
from sklearn.linear_model import LinearRegression

model = LinearRegression()

# モデルの学習
model.fit(X, Y)

# coefficient: 係数
# intercept  : 切片
print('coefficient={:.3f}, intercept={:.3f}'.format(model.coef_[0], model.intercept_))


Y_predict = model.predict(X)

# 決定係数
from sklearn.metrics import r2_score
print('R^2={:.3f}'.format(r2_score(Y, Y_predict)))

coefficient=2.777, intercept=4.693
R^2=0.847


In [18]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc

tip_median = data['tip'].median()


X = data[['distance', 'color', 'payment', 'pickup_zone', 'dropoff_zone']]
X = pd.get_dummies(data=X, drop_first=True)
Y = data.tip.map(lambda x: 1 if x >= tip_median else 0)

model2 = LogisticRegression(C=10000.0, max_iter=10000000)

# 学習
model2.fit(X, Y)

Y_proba = model2.predict_proba(X)
fpr, tpr, thresholds = roc_curve(Y, Y_proba[:, 1])
print('ROC曲線: AUC={:.3f}'.format(auc(fpr, tpr)))

ROC曲線: AUC=0.919


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, auc
from sklearn.model_selection import cross_val_score

# 途中です

def classify(depth):
    # 決定木を作成
    model = DecisionTreeClassifier(random_state=0, criterion='entropy', max_depth=depth)

    # 交差検証の実行
    score = cross_val_score(model, X, Y, cv=5, scoring='roc_auc')
    print('depth={:2}: AUC={:.3f} (+/- {:.3f})'.format(depth, score.mean(), score.std()))

for d in range(1, 11, 1):
    classify(d)

### 2.2.2 階層的クラスタ分析

2.2.2 では以下（樹形図とヒートマップ付きの樹形図とは別々に作成する）を行うこと。必要ならば他の処理を記載しても構わない。

- 樹形図（デンドログラム）の作成
- ヒートマップ付きの樹形図の作成

In [4]:
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt

# ランク系は取り除く
# 所属クラブ以外の要素によるクラスタリングがしたいのでClubも取り除く
# 名前　リーグ　シーズンは取り除く
# ポジション　ダミー変数化
drop_target = ['Current Club', 'rank_in_league_top_attackers', 'rank_in_league_top_midfielders',
       'rank_in_league_top_defenders', 'rank_in_club_top_scorer']

player.columns
# X = player.drop(drop_target, axis=1)


# clusters = linkage(X.values, method='single', metric='euclidean')

# # 樹形図を作成
# plt.figure(figsize=(50, 20))
# dendr = dendrogram(clusters, labels=X.index, leaf_font_size=14)

Index(['full_name', 'age', 'birthday', 'birthday_GMT', 'league', 'season',
       'position', 'Current Club', 'minutes_played_overall',
       'minutes_played_home', 'minutes_played_away', 'nationality',
       'appearances_overall', 'appearances_home', 'appearances_away',
       'goals_overall', 'goals_home', 'goals_away', 'assists_overall',
       'assists_home', 'assists_away', 'penalty_goals', 'penalty_misses',
       'clean_sheets_overall', 'clean_sheets_home', 'clean_sheets_away',
       'conceded_overall', 'conceded_home', 'conceded_away',
       'yellow_cards_overall', 'red_cards_overall',
       'goals_involved_per_90_overall', 'assists_per_90_overall',
       'goals_per_90_overall', 'goals_per_90_home', 'goals_per_90_away',
       'min_per_goal_overall', 'conceded_per_90_overall',
       'min_per_conceded_overall', 'min_per_match', 'min_per_card_overall',
       'min_per_assist_overall', 'cards_per_90_overall',
       'rank_in_league_top_attackers', 'rank_in_league_top_midfie

### 2.2.3 非階層的クラスタ分析

2.2.3 では以下を行うこと。必要ならば他の処理を記載しても構わない。

- クラスタ数の推定
  - エルボー法
  - シルエット分析
- KMeans法によるクラスタ分析
  - クラスタ数の設定については理由を記載すること
  - クラスタ分析の結果は可視化も用いて説明すること

### 2.2.4 考察

- 階層的、非階層的クラスタ分析の結果について考察